- このノートブックの内容は2024年5月時点の [LangChain 公式ドキュメント](https://python.langchain.com/v0.1/docs/get_started/introduction)に依拠しています。
- 事前に社内規定PDFを概要欄リンクからダウンロードし、動画内の説明に従ってアップロードしておいてください。

## 準備：API キーの設定

In [ ]:
!pip install -q openai
!pip install -q pypdf
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q langchain-chroma
!pip install -q docx2txt

In [ ]:

from google.colab import userdata
import openai

# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = openai.api_key

# RAG（LLM の知識拡張）
RAG を活用することで、LLM は学習データに含まれない知識を得ることができます。

参照用データとしては、現在 LangChain は以下に対応しています：
- .txt
- .csv
- .html
- .json
- .md
- .pdf
- MS Office ファイル

今回は PDF からの知識抽出を試してみましょう。

In [ ]:
#from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("docx/example.docx")
data = loader.load()


In [ ]:
data

[Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月11\n\n日\n\n報 告 期 間\n\n2024年6月03日 ～ 2024年6月07\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000□05/28) OperationStaffMeeting（6件）\n\n\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\n\n\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\n\n\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\n\n\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \n\n\u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\n\n\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\n\n\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\n\n\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒

In [ ]:
len(data)

1

## 文埋め込みと類似文書検索

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
from langchain_chroma import Chroma

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

In [ ]:
# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})

In [ ]:
query = "VESPA お問合せの内容は？"

retriever.invoke(query)

[Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月11\n\n日\n\n報 告 期 間\n\n2024年6月03日 ～ 2024年6月07\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000□05/28) OperationStaffMeeting（6件）\n\n\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\n\n\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\n\n\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\n\n\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \n\n\u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\n\n\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\n\n\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\n\n\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒

## Retrieval を活用したチャットモデルの制御

### まずはただ聞いてみる

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
gpt4o = ChatOpenAI(model="gpt-4o", temperature=0.1)
out = gpt4o.invoke("VESPAお問合せの内容は？")
print(out.content)

「VESPAお問合せの内容は？」という質問に対して、具体的な情報が不足しているため、一般的な回答を提供します。VESPAに関するお問い合わせ内容は多岐にわたる可能性があります。以下は、一般的な問い合わせ内容の例です：

1. **製品情報**:
   - 新しいモデルの詳細
   - 技術仕様
   - カラーバリエーション
   - 価格

2. **購入に関する質問**:
   - 購入方法
   - 支払いオプション
   - ディーラーの所在地
   - 試乗の予約

3. **アフターサービス**:
   - 保証期間と内容
   - メンテナンスサービス
   - 修理の予約
   - 部品の取り寄せ

4. **カスタマイズ**:
   - アクセサリーのオプション
   - カスタムペイントやデザイン

5. **技術サポート**:
   - トラブルシューティング
   - 使用方法のガイド
   - ソフトウェアアップデート（電動モデルの場合）

6. **その他**:
   - イベント情報
   - プロモーションやキャンペーン
   - 環境への取り組み

具体的な問い合わせ内容に応じて、VESPAの公式ウェブサイトやカスタマーサポートに連絡することをお勧めします。


### 準備：プロンプトテンプレートを作る

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)

In [ ]:
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever.invoke("VESPA お問合せの内容は？")

[Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月11\n\n日\n\n報 告 期 間\n\n2024年6月03日 ～ 2024年6月07\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000□05/28) OperationStaffMeeting（6件）\n\n\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\n\n\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\n\n\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\n\n\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \n\n\u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\n\n\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\n\n\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\n\n\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒

In [ ]:
retrieved = retriever.invoke("VESPA お問合せの内容は？")

print(retrieved[0].page_content)

進 捗 状 況 報 告 書（週報）

システム名

																SDR、VDA、MAC



2024年6月11

日

報 告 期 間

2024年6月03日 ～ 2024年6月07

日

作 成 者

石岡雅紀

当



期



作



業



実



施

工程

SDR、VDA、PSPM, DTC Priority

作業項目

SDR、VDA、PSPM, DTC PriorityのAA担当



赤字：待ち

緑字：対応予定

---------------------------------------------------------------------------------------------------

□　業務部署（GAS／XX6等）問合せ／周知　　

　　□05/28) OperationStaffMeeting（6件）

　　　　　⇒■05/31) GAS展開、ADT Nissan team週次会議　

　　　　　※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中

　　　06/04) OperationStaffMeeting（1件）

　　　　　⇒　AMO確認待ち 

　　□GAS問合せ（依頼：証明書期限一覧の作成）

　　　　⇒　■04/01) 松ケ野さん、正英さんレビュー　内部向けの資料としては完了

　　　　⇒　GAS展開用の資料作成中

　　□VESPA問合せ（依頼：VESPAの認証方式変更　IDP⇒OKTA）

　　　　⇒　■04/24) 保守定例内で開発案件となる方向であると確認。またIBXの他部署とも対応方針をそろえた方が良いので、情報収集が必要。

　　　　⇒　■04/25）VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメール受領

　　　　⇒　■05/09）IB1伊藤-3より、既に1月にOKTAに変更されている旨回答有り

　　　　⇒　AMOにてIB3も対応不要で問題ないか確認中　configの修正対応？

　　　　⇒　保守開発間で返答内容を取りまとめてアンケート回答が必要

　　　　※接続テストの経費が必要になりそう

　　□XX6問合せ（依頼：CDGのレスポンスの遅れにより、CONSULT側でタイムアウト

### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [ ]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

In [ ]:
text_formatter(retrieved)

'進 捗 状 況 報 告 書（週報）システム名\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC2024年6月11日報 告 期 間2024年6月03日 ～ 2024年6月07日作 成 者石岡雅紀当期作業実施工程SDR、VDA、PSPM, DTC Priority作業項目SDR、VDA、PSPM, DTC PriorityのAA担当赤字：待ち緑字：対応予定---------------------------------------------------------------------------------------------------□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\u3000\u3000□05/28) OperationStaffMeeting（6件）\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒OKTA）\u3000\u3000\u3000\u3000⇒\u3000■04/24) 保守定例内で開発案件となる方向であると確認。またIBXの他部署とも対応方針をそろえた方が良いので、情報収集が必要。\u3000\u3000\u3000\u3000⇒\u3000■04/25）VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメール受領\u30

### 抽出・補完・生成のチェインを作る

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [ ]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [ ]:
chain.invoke("VESPA お問合せの内容は？")

AIMessage(content='VESPAに関する問い合わせの内容は以下の通りです：\n\n1. **VESPAの認証方式変更 (IDP⇒OKTA)**\n   - **04/24**: 保守定例内で開発案件となる方向で確認。また、IBXの他部署とも対応方針をそろえた方が良いので情報収集が必要。\n   - **04/25**: VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメールを受領。\n   - **05/09**: IB1伊藤-3より、既に1月にOKTAに変更されている旨の回答あり。\n   - **AMO**にてIB3も対応不要で問題ないか確認中（configの修正対応が必要かどうかも含む）。\n   - **保守開発間**で返答内容を取りまとめてアンケート回答が必要。\n   - **接続テストの経費**が必要になりそう。\n\nこれらの内容から、VESPAの認証方式をIDPからOKTAに変更する件についての問い合わせが行われており、それに伴う対応状況の確認や調整が必要であることが記載されています。', response_metadata={'token_usage': {'completion_tokens': 277, 'prompt_tokens': 5891, 'total_tokens': 6168}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-48b06527-8bb7-4b4f-b624-52d7ee0b65b4-0')

In [ ]:
resp=chain.invoke("全体的に問題ポイントを教えて")

In [ ]:
resp.content

'進捗状況報告書の内容から、問題点や改善が必要なポイントを以下に整理します：\n\n### 1. **未解決の課題や対応待ちが多い**\n- **GAS問合せ**：PSPMが対象リストに含まれていなかった理由が未確認（AMO確認待ち）。\n- **VESPA問合せ**：アンケートの返答が未完了（AMO確認待ち）。\n- **XX6問合せ**：CPU高騰の原因調査が未完了（AMO確認待ち）。\n- **SDRでのH15 Key登録エラー**：Daimlerからの返信待ち。\n- **TLS1.0無効化対応**：6/10の動確完了連絡待ち。\n- **BCVデータ重複対応**：恒久対応後の進め方が未確定（業連の調整中）。\n\n### 2. **連絡や確認待ちが多い**\n- **AMO確認待ち**：複数の項目でAMOからの確認や回答待ちが多い。\n- **富士通AMOからの連絡待ち**：SDRログ容量増加対応、PSPM Cookie影響調査、BCVデータ重複対応など。\n\n### 3. **調整や確認に時間がかかっている**\n- **業連調整**：業連の発行や調整が複数あり、進捗が遅れている。\n- **VESPAリプロチェック**：リリース後の動確がまだ完了していない。\n\n### 4. **明確なスケジュールや期限が不明**\n- 各タスクや対応の完了予定日が明確に示されていないため、進捗管理が難しい可能性がある。\n\n### 5. **重複したタスクや対応の優先順位が不明確**\n- 複数の重複したタスクが存在し、どのタスクが優先されるべきかが明確ではない。\n\n### 6. **コミュニケーション不足**\n- 幾つかの項目で連絡が遅れていたり、返答がないまま時間が経過している。\n- 例：監視アラートの連絡先変更が未完了。\n\n### 7. **費用の扱いや予算調整が不明確**\n- 費用の扱いや予算調整に関して、具体的な計画や進捗が見られない。\n\n### 8. **障害対応の再発防止策が不明確**\n- 障害の原因調査や再発防止策についての具体的な対応策が明示されていない。\n\n### 改善の提案\n1. **進捗の可視化**：未解決の課題や対応待ちの項目を一覧化し、進捗状況を定期的に更新・共有する。\n2. **期限設定**：各タスクや

In [ ]:
resp=chain.invoke("VESPAリプロチェック　リリース後の動作確認は完了してますか？")

In [ ]:
resp.content

'「VESPAリプロチェック\u3000リリース後の動作確認未完了」という記述は、システムやソフトウェアの変更や更新が行われた後、その変更が正しく機能するかどうかを確認するためのテスト（動作確認）がまだ完了していないことを意味します。具体的には以下のような状況が考えられます：\n\n1. **リリース後のテストが未完了**：ソフトウェアの新しいバージョンや修正が本番環境にリリースされた後、その動作が期待通りであるかどうかの検証がまだ行われていない。\n\n2. **確認作業の遅延**：動作確認を担当するチームや個人が、他のタスクやスケジュールの都合でまだ確認作業を完了していない。\n\n3. **問題発見の可能性**：動作確認が完了していないため、リリースされた変更に何らかの問題や不具合が潜在的に含まれている可能性がある。\n\nこのため、「リリース後の動作確認未完了」とは、リリースされたソフトウェア変更が安定して動作することを確認するプロセスがまだ完了していないため、潜在的なリスクが残っている状態を示しています。\n\n具体的な進捗状況としては、2024年6月10日に「リリース後の動確完了」となっているため、その時点で動作確認が完了していることが報告されています。したがって、現時点では動作確認が完了し、問題がないことが確認されたことになります。'

In [ ]:
resp=chain.invoke("全体的に問題ポイントに「VESPAリプロチェック　リリース後の動作確認未完了」はどいう意味でしょうか？")